THis is  a page to write a script to extract notice information from reginfo.gov icr pages and submission date

In [59]:
import requests
from bs4 import BeautifulSoup 
import csv
import datetime 
from csv import reader
import os 
import pandas as pd 
import pycurl
import time
import glob
import sqlite3

In [2]:
os.getcwd()

'/home/jupyter-ed/projects/pra/scripts/EDA'

In [3]:
os.listdir()

['notices_1235-0015.json',
 '.ipynb_checkpoints',
 'icr_numbers.csv',
 'pending_eda.ipynb',
 'notice_2020-07-29',
 'html',
 'icr_html_noticedate.ipynb',
 'history_eda.ipynb',
 'connect_to_postgres_database_and_add_a_df_as_table.ipynb',
 'todo_for_open_loop_report.ipynb',
 'notice_2020-07-28']

In [4]:
os.chdir('/home/jupyter-ed/projects/pra/scripts/EDA')

In [5]:


# generate the list of links
def link_slug_add(link):
  pagelink = 'https://www.reginfo.gov/public/do/PRAViewICR?ref_nbr='+str.strip(link)
  return(pagelink)
  
    
def icr_link_list_creator(icr_number_list):
    icr_url_list = []
    for number in icr_number_list:
        reg_info_url = link_slug_add(number)
        icr_url_list.append(reg_info_url)    
    return(icr_url_list)
    
# create the new folder for the days html files

def create_icr_html_folder():
    folder_name = datetime.datetime.now().strftime('icr_folder_%Y-%m-%d')   
    os.chdir('/home/jupyter-ed/projects/pra/data/history')
    # define the name of the directory to be created
    os.mkdir(folder_name)
    os.chdir(folder_name)
     
# download all the files to the folder
def get_icr_html_file(url,icr_number):
    filename = icr_number +'.html'    
    file = open(filename,'wb')
    crl = pycurl.Curl()
    crl.setopt(crl.URL, url)
    crl.setopt(crl.WRITEDATA, file)
    crl.perform()
    crl.close()

def hmtl_down_from_cntrl_number_list(omb_cntrl_number_list,omb_history_url_list):
    list_range = range(len(omb_cntrl_number_list))
    for number in list_range:
        omb_cntrl_instance = omb_cntrl_number_list[number]
        praurl = omb_history_url_list[number]
        get_icr_html_file(praurl,omb_cntrl_instance) 
        time.sleep(3)
    
    
    

csv_file = 'icr_numbers.csv'

icr_number_csv = pd.read_csv(csv_file)


icr_number_csv.columns

icr_number_list = list(icr_number_csv.icr_reference_number.values)
    
    
icr_url_list = icr_link_list_creator(icr_number_list)   
    
print('url list created')


print(os.getcwd)     
create_icr_html_folder()    
print(os.getcwd) 


   
    

    

     
    
    
hmtl_down_from_cntrl_number_list(icr_number_list,icr_url_list)    

print('html files downloaded') 

In [6]:
# going to move files into the html folder in the eda folder

In [7]:
#jupyter-ed@jupyterhubtestdaily:~/projects/pra/scripts/EDA/html$ mv *.html /home/jupyter-ed/projects/pra/scripts/EDA/html# 

In [8]:
folder_for_html = '/home/jupyter-ed/projects/pra/scripts/EDA/html'

In [9]:
os.chdir(folder_for_html)

In [10]:
html_files = os.listdir()

In [11]:
html_file= html_files[0]

In [12]:
html_file

'201809-1219-005.html'

In [13]:
with open(html_file, 'r') as f:

    contents = f.read()

    soup = BeautifulSoup(contents, 'lxml')

    #print(soup.h2)
    #print(soup.head)
    ombcntrl_table = soup.find('div',{'id':'section5'})
    #print(ombcntrl_table)
    rows = ombcntrl_table.findAll('tr')
    #print(len(rows))
    notice_60_day = rows[2]
    #print(notice_60_day)
    cells = notice_60_day.findAll('td')
    print(len(cells))
    #print(cells)
    print(cells[1].getText(strip='True'))
    print(cells[2].getText(strip='True'))
    #for cell in cells:
    #    try:
    #        print(cell.getText())
    #    except:
    #        print("no text")

3
83 FR 33252
07/17/2018


In [14]:
def get_notice_date(html_file):
    with open(html_file, 'r') as f:
        contents = f.read()
        soup = BeautifulSoup(contents, 'lxml')
        section5 = soup.find('div',{'id':'section5'})
        rows = section5.findAll('tr')
        row_60_day_notice = rows[2]
        cells = row_60_day_notice.findAll('td')
        data_row = [html_file,cells[1].getText(strip='True'),cells[2].getText(strip='True')]
        return(data_row)

In [15]:
test = get_notice_date(html_file)

In [16]:
print(test)

['201809-1219-005.html', '83 FR 33252', '07/17/2018']


In [18]:
#def get_notice_date(html_file):
with open(html_file, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    section1 = soup.find('div',{'id':'section1'})
    table = section1.find('table',{'class':'generalTxt'})
    rows = table.findAll('tr')
    row_60_day_notice = rows[2]
    cells = row_60_day_notice.findAll('td')
    data_row = [html_file,cells[1].getText(strip='True'),cells[2].getText(strip='True')]
    print(data_row)

IndexError: list index out of range

In [19]:
#def get_notice_date(html_file):
list_of_rows = []

with open(html_file, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    section1 = soup.find('div',{'id':'section1'})
    table = section1.find('table',{'class':'generalTxt'})
    rows = table.findAll('tr')
    print(len(rows))
    for row in rows:
        list_of_rows.append(row.findAll('td'))    
        

9


In [20]:
len(list_of_rows)

9

In [21]:
list_of_rows[1][1].getText()

'\nPrevious ICR Reference No:\n201406-1219-005\n'

In [22]:
for thing in list_of_rows:
    print(len(thing))

2
2
2
1
2
2
2
2
1


In [23]:
row1 

NameError: name 'row1' is not defined

In [24]:
#<label>OMB Control No:</label>
list_of_rows[0][0].getText(strip='True')

'OMB Control No:1219-0088'

In [25]:
#<label>ICR Reference No:</label>
list_of_rows[0][1].getText(strip='True')

'ICR Reference No:201809-1219-005'

In [26]:
#<label>Status:</label>
list_of_rows[1][0]

<td width="50%">
<label>Status:</label>
																				Active
																			</td>

<label>Previous ICR Reference No:</label>

In [27]:
list_of_rows[1][1]

<td width="50%">
<label>Previous ICR Reference No:</label>
<a aria-label="Previous ICR Reference No" href="/public/do/PRAViewICR?ref_nbr=201406-1219-005">201406-1219-005</a>
</td>

In [28]:
list_of_rows[2][0]

<td width="50%">
<label>Agency/Subagency:</label>
																				DOL/MSHA
																			</td>

In [29]:
list_of_rows[2][1]

<td width="50%">
<label>Agency Tracking No:</label>
</td>

In [30]:
list_of_rows[3][0]

<td colspan="2">
<label>Title:</label>
																			    Ventilation Plans, Tests, and Examinations in Underground Coal Mines
																			</td>

In [31]:
list_of_rows[4][0]

<td width="50%">
<label>Type of Information Collection:</label>
																				Extension without change of a currently approved collection
																			</td>

In [32]:
list_of_rows[4][1]

<td width="50%">
<label>Common Form ICR:</label> 
																				
																		        
																		           No
																		        
																			</td>

In [33]:
list_of_rows[5][0]

<td width="50%">
<label>Type of Review Request:</label>
																				Regular
																			</td>

In [34]:
list_of_rows[5][1]

<td width="50%">
</td>

In [35]:
list_of_rows[6][0]

<td width="50%">
<label>OIRA Conclusion Action:</label>
																					Approved without change
																			</td>

In [36]:
list_of_rows[6][1]

<td width="50%">
<label>Conclusion Date:</label>
																				02/12/2019
																			</td>

In [37]:
list_of_rows[7][0]

<td width="50%">
<a href="#" onclick="openNOA(292618)">Retrieve Notice of Action (NOA)</a>
</td>

In [38]:
#<label>Date Received in OIRA: </label>

list_of_rows[7][1].getText(strip='True')

'Date Received in OIRA:11/30/2018'

In [39]:
list_of_rows[8][0]

<td colspan="2">
<label>Terms of Clearance:</label>
</td>

All i need right now is the 60 day notice date, the submission , ombcontrol and the icr number

In [40]:
def get_section1_info_list(html_file):
    list_of_rows = []
    
    with open(html_file, 'r') as f:
        contents = f.read()
        soup = BeautifulSoup(contents, 'lxml')
        section1 = soup.find('div',{'id':'section1'})
        table = section1.find('table',{'class':'generalTxt'})
        rows = table.findAll('tr')
        for row in rows:
            list_of_rows.append(row.findAll('td'))   
    return(list_of_rows)   
    

In [41]:
def section1_info_extractor(list_of_rows):
        #<label>OMB Control No:</label>
        omb_control_number = list_of_rows[0][0].getText(strip='True')
        #<label>ICR Reference No:</label>
        icr_reference_number = list_of_rows[0][1].getText(strip='True')        
        #<label>Date Received in OIRA: </label>
        submited_to_oira_date = list_of_rows[7][1].getText(strip='True') 
        previous_icr = list_of_rows[1][1].getText(strip='True')
        section1_list = [omb_control_number,icr_reference_number,submited_to_oira_date,previous_icr]
        return(section1_list)
        

In [42]:
def get_notice_date(html_file):
    with open(html_file, 'r') as f:
        contents = f.read()
        soup = BeautifulSoup(contents, 'lxml')
        section5 = soup.find('div',{'id':'section5'})
        rows = section5.findAll('tr')
        row_60_day_notice = rows[2]
        cells = row_60_day_notice.findAll('td')
        data_row = [cells[1].getText(strip='True'),cells[2].getText(strip='True')]
        return(data_row)

In [43]:
list_of_rows = get_section1_info_list(html_file)
section1_list = section1_info_extractor(list_of_rows)
notice_data_row = get_notice_date(html_file)
section1_list.extend(notice_data_row)




In [44]:
section1_list

['OMB Control No:1219-0088',
 'ICR Reference No:201809-1219-005',
 'Date Received in OIRA:11/30/2018',
 'Previous ICR Reference No:201406-1219-005',
 '83 FR 33252',
 '07/17/2018']

In [45]:
section1_list[0].split(':')[1]

'1219-0088'

'OMB Control No:,'ICR Reference No: 'Date Received in OIRA:, citiation date, publication date

In [46]:
[section1_list[0].split(':')[1],
 section1_list[1].split(':')[1],
 section1_list[2].split(':')[1],
 section1_list[3].split(':')[1],
 section1_list[4],
 section1_list[5]]

['1219-0088',
 '201809-1219-005',
 '11/30/2018',
 '201406-1219-005',
 '83 FR 33252',
 '07/17/2018']

pusedo code

for each html file
create the data row
add the data row to a list


create pandas dataframe
insert into sqlitedb













In [47]:
def icr_page_info_extractor(html_file):
    list_of_rows = get_section1_info_list(html_file)
    section1_list = section1_info_extractor(list_of_rows)
    notice_data_row = get_notice_date(html_file)
    section1_list.extend(notice_data_row)
    icr_notice_data_row = [section1_list[0].split(':')[1],
                           section1_list[1].split(':')[1],
                           section1_list[2].split(':')[1],
                           section1_list[3].split(':')[1],
                           section1_list[4],
                           section1_list[5]]
    return(icr_notice_data_row)
    




In [50]:
os.getcwd()

'/home/jupyter-ed/projects/pra/scripts/EDA/html'

In [51]:
files = glob.glob("*.html")


In [52]:
files

['201809-1219-005.html',
 '201803-1218-001.html',
 '201902-1218-001.html',
 '201706-1219-004.html',
 '201803-1225-001.html',
 '201709-1218-004.html',
 '201802-1218-005.html',
 '201611-1240-005.html',
 '201611-1240-004.html',
 '201809-1218-023.html',
 '201707-1218-005.html',
 '201809-1218-001.html',
 '201911-1205-003.html',
 '201906-1210-003.html',
 '201902-1225-001CF.html',
 '201811-1218-002.html',
 '202003-1205-004.html',
 '201809-1219-003.html',
 '201807-1205-001.html',
 '202004-1205-001.html',
 '202006-1220-005.html',
 '202003-1240-003.html',
 '201901-1210-005.html',
 '201805-1290-002.html',
 '201707-1218-002.html',
 '202002-1210-004.html',
 '202004-1245-001.html',
 '201805-1210-004.html',
 '201903-1218-004.html',
 '201611-1240-002.html',
 '201808-1205-003.html',
 '202004-1220-004.html',
 '202002-1205-002.html',
 '201910-1218-003.html',
 '201912-1218-009.html',
 '201804-1290-001.html',
 '201812-1220-002.html',
 '201909-1205-006.html',
 '201612-1235-001.html',
 '201907-1205-002.html'

In [53]:
icr_notice_data_list = []

In [54]:
for file in files:
    try:
        icr_notice_data_list.append(icr_page_info_extractor(file))
    except:
        icr_notice_data_list.append(['x','x','x','x','x'])
    print(len(icr_notice_data_list))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [55]:
files[15]

'201811-1218-002.html'

In [56]:
len(icr_notice_data_list[14])

5

In [57]:
icr_notice_data_list[15]

['1218-0096',
 '201811-1218-002',
 '03/01/2019',
 '201507-1218-002',
 '83 FR 47190',
 '09/18/2018']

In [66]:
os.getcwd()

'/home/jupyter-ed/projects/pra/scripts/EDA/html'

In [67]:
os.chdir('/home/jupyter-ed/projects/pra/data/01_database')

In [68]:
DBPATH = 'pradb.db'
conn = sqlite3.connect(DBPATH)
cursor = conn.cursor()

In [69]:
icr_notice_data_df = pd.DataFrame(icr_notice_data_list)

In [70]:
icr_notice_data_df.columns = ['omb_control_number',
                              'icr_reference_control_number',
                              'submission_date',
                              'prev_icr',
                              'citation',
                              'publication_date']

In [71]:
icr_notice_data_df.to_sql('icr_reginfo_data',con=conn,if_exists='replace')

In [ ]:
notice_dataframe.to_sql('notices',con=conn,if_exists='replace')

In [ ]:


#https://www.reginfo.gov/public/do/PRAViewICR?ref_nbr=
# here are all the functions to get the info


# create the reginfo link from the omb control number
def link_slug_add(link):
  pagelink = 'https://www.reginfo.gov/public/do/PRAViewICR?ref_nbr='+str.strip(link)
  return(pagelink)


# test the link 
def link_checker(link):
  if type(link)==str:
    if len(link) > 50:
      return(link)
    else:
      print('bad link')


# get the html webpage from the link
# use the tr td to filter for the info

def get_page(link):
  page = requests.get(link).content
  sel = Selector(text = page)
  trpath = '//tr//td//text()'
  table =sel.xpath(trpath).extract()
  return(table)


# use the output from the function above to get the 
# information about the icr

def omb_info_builder(page,link):
  for number in range(0,len(page)):
     webpage_string =str.strip(page[number])
     if webpage_string =="OMB Control No:":
       omb_info = {}
       omb_info['link'] = link
       omb_info['OMB Control No']= str.strip(page[number+1])
       omb_info['ICR Reference No']= str.strip(page[number+5])
       omb_info['Status']= str.strip(page[number+11])
       omb_info['Previous ICR Reference No']= str.strip(page[number+16])
       omb_info['Agency/Subagency']= str.strip(page[number+23])
       omb_info['Title']= str.strip(page[number+33])
       omb_info['Type of Information Collection']= str.strip(page[number+39])
       omb_info['Common Form ICR']= str.strip(page[number+43])
       omb_info['Type of Review Request']= str.strip(page[number+49])
       omb_info['OIRA Conclusion Action']= str.strip(page[number+57])
       omb_info['Conclusion Date']= str.strip(page[number+61])
       omb_info['Date Received in OIRA']= str.strip(page[number+71])
       omb_info['check_if_new'] = str.strip(page[336])
       return(omb_info)

# uses the same html to find the frn dates and citations

def frn_date_builder(page):
  for number in range(0,len(page)):
     webpage_string =str.strip(page[number])
     if webpage_string =="Federal Register Notices & Comments":
       frn_info = {"link":link,
       "60-day_date": str.strip(page[number+25]),
       "30_day_date": str.strip(page[number+49]),
       "60_day_citation":str.strip(page[number+22]),
       "30_day_citation": str.strip(page[number+46])}
       return(frn_info)


# uses the same html to find the expiration date of the icr 

def expiration_date_scrape(page,link):
  for number in range(0,len(page)):
    webpage_string =str.strip(page[number])
    if webpage_string =="Expiration Date":
      exp_info = {"link":link,"expiration_date ":str(page[number+2]),"prev_expiration_date ":str(page[number+6])}
      return(exp_info)


# read in the list of active icr omb control numbers

os.chdir('/home/ed/projects/pra/improve/data')

cntrl_numbers = 'omb_cntr_numbers_inventory_2020_07_14.csv'

with open(cntrl_numbers, 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Pass reader object to list() to get a list of lists
    list_of_rows = list(csv_reader)

del list_of_rows[0]

all_links = []

# create the reginfo.gov links from the omb control numbers

for row in list_of_rows:
  if len(row) > 0 :
    link = link_slug_add(row[0])
    all_links.append(link)
  

       
good_links = [ ]


# test the links

for link in all_links:
  checked_link =link_checker(link)
  test1 = checked_link != 'bad link'
  test2 = type(checked_link)==str
  if (test1==True and test2==True):
    good_links.append(checked_link)
    

# create a place to save the icr info from each function

frn_list = []
omb_info = []
expirations = []


  
# get the page, then get the info, save the info to the lists 
# from above

for link in good_links:
  page = get_page(link)
  frn_data = frn_date_builder(page)
  omb_data = omb_info_builder(page,link)
  exp = expiration_date_scrape(page,link)
  frn_list.append(frn_data)
  omb_info.append(omb_data)
  expirations.append(exp)
  print(link)
  
  
  

frn_csv_columns = list(frn_list[1].keys())
now = datetime.now()
t_string = now.strftime("%Y%H%M%S")
csv_file = t_string+"_frn.csv"

try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=frn_csv_columns)
        writer.writeheader()
        for data in frn_list:
            writer.writerow(data)
except IOError:
    print("I/O error")



omb_csv_columns = list(omb_info[1].keys())
now = datetime.now()
t_string = now.strftime("%Y%H%M%S")

csv_file = t_string+"_omb.csv"

try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=omb_csv_columns)
        writer.writeheader()
        for data in omb_info:
            writer.writerow(data)
except IOError:
    print("I/O error")  
  
  
exp_csv_columns = list(expirations[1].keys())
now = datetime.now()
t_string = now.strftime("%Y%-%m-%d-%H-%S")

csv_file = t_string+"_exp.csv"

try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=exp_csv_columns)
        writer.writeheader()
        for data in expirations:
            writer.writerow(data)
except IOError:
    print("I/O error") 